<a href="https://colab.research.google.com/github/QMSforever/CS229/blob/main/%E2%80%9CCS_229_Transformers_in_Pytorch_ipynb%E2%80%9DCopy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Remember to make a copy of this colab notebook before you start editing cells!

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 7.1 MB/s eta 0:00:00


In [ ]:
!pip install tqdm

In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-02-20 02:30:36--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2025-02-20 02:30:36 (29.7 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
# DO NOT MODIFY ANY OF THIS CODE

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm

In [ ]:
# DO NOT MODIFY ANY OF THIS CODE

# Hyperparameters
batch_sz = 16
context_length = 32
max_iterations = 30000
log_interval = 200
init_lr = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_steps = 200
embedding_dim = 64
num_heads = 4
num_blocks = 4
drop_prob = 0.0

# Load and prepare the data
torch.manual_seed(1337)
with open('input.txt', 'r', encoding='utf-8') as file:
    text_data = file.read()

unique_chars = sorted(set(text_data))
vocab_size = len(unique_chars)
char_to_index = {ch: i for i, ch in enumerate(unique_chars)}
index_to_char = {i: ch for i, ch in enumerate(unique_chars)}

def encode_text(s): return [char_to_index[c] for c in s]
def decode_text(l): return ''.join([index_to_char[i] for i in l])

# Split data for training and validation
data_tensor = torch.tensor(encode_text(text_data), dtype=torch.long)
train_size = int(0.9 * len(data_tensor))
train_data, val_data = data_tensor[:train_size], data_tensor[train_size:]

def generate_batch(split):
    data_src = train_data if split == 'train' else val_data
    indices = torch.randint(0, len(data_src) - context_length, (batch_sz,))
    inputs = torch.stack([data_src[i:i + context_length] for i in indices])
    targets = torch.stack([data_src[i + 1:i + context_length + 1] for i in indices])
    return inputs.to(device), targets.to(device)

@torch.no_grad()
def evaluate_loss():
    model.eval()
    losses = {'train': [], 'val': []}
    for split in ['train', 'val']:
        for _ in range(eval_steps):
            batch_x, batch_y = generate_batch(split)
            _, batch_loss = model(batch_x, batch_y)
            losses[split].append(batch_loss.item())
    model.train()
    return {split: torch.tensor(losses[split]).mean().item() for split in losses}

In [ ]:
# YOU WILL CHANGE CODE IN THIS CELL
# Implement a Transformer model with PyTorch. Fill out the provided skeleton.

class SelfAttention(nn.Module):
    def __init__(self, head_dim):
        super().__init__()
        # Initialize key, query, and value linear layers without bias.
        self.key_proj = nn.Linear(embedding_dim, head_dim, bias=False)
        self.query_proj = nn.Linear(embedding_dim, head_dim, bias=False)
        self.value_proj = nn.Linear(embedding_dim, head_dim, bias=False)
        # Lower triangular mask to ensure causal attention.
        self.register_buffer('mask', torch.tril(torch.ones(context_length, context_length)))
        self.dropout = nn.Dropout(drop_prob)

    def forward(self, x):
        # x: (B, T, C)
        B, T, C = x.shape
        # Project input x to keys, queries, and values.
        keys = self.key_proj(x)      # (B, T, head_dim)
        queries = self.query_proj(x) # (B, T, head_dim)
        values = self.value_proj(x)  # (B, T, head_dim)
        # Scale dot-product attention.
        scale = queries.size(-1) ** -0.5  # using head_dim for scaling
        scores = (queries @ keys.transpose(-2, -1)) * scale  # (B, T, T)
        # Apply causal mask.
        scores = scores.masked_fill(self.mask[:T, :T] == 0, float('-inf'))
        # Apply softmax to get attention weights and then dropout.
        attention_weights = F.softmax(scores, dim=-1)
        attention_weights = self.dropout(attention_weights)
        # Compute weighted sum of values.
        return attention_weights @ values  # (B, T, head_dim)


class MultiHeadSelfAttention(nn.Module):
    def __init__(self, num_heads, head_dim):
        super().__init__()
        # Create a list of self attention heads.
        self.heads = nn.ModuleList([SelfAttention(head_dim) for _ in range(num_heads)])
        # Output projection to combine heads.
        self.output_proj = nn.Linear(num_heads * head_dim, embedding_dim)
        self.dropout = nn.Dropout(drop_prob)

    def forward(self, x):
        # Apply each attention head.
        head_outputs = [head(x) for head in self.heads]  # Each: (B, T, head_dim)
        # Concatenate outputs along the last dimension.
        x = torch.cat(head_outputs, dim=-1)  # (B, T, num_heads * head_dim)
        # Apply final projection and dropout.
        return self.dropout(self.output_proj(x))


class FeedForwardLayer(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(emb_dim, 4 * emb_dim),
            nn.ReLU(),
            nn.Linear(4 * emb_dim, emb_dim),
            nn.Dropout(drop_prob)
        )

    def forward(self, x):
        return self.layers(x)


class TransformerBlock(nn.Module):
    def __init__(self, emb_dim, num_heads):
        super().__init__()
        head_dim = emb_dim // num_heads  # derive head dimension from embedding dimension and number of heads
        # Initialize multi-head self attention, feed-forward layer, and layer norms.
        self.attention = MultiHeadSelfAttention(num_heads, head_dim)
        self.feed_forward = FeedForwardLayer(emb_dim)
        self.norm1 = nn.LayerNorm(emb_dim)
        self.norm2 = nn.LayerNorm(emb_dim)

    def forward(self, x):
        # Apply layer norm followed by self attention with a residual connection.
        x = x + self.attention(self.norm1(x))
        # Apply second layer norm followed by feed forward layer with residual connection.
        x = x + self.feed_forward(self.norm2(x))
        return x


class TransformerLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # Token and positional embeddings.
        self.token_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.position_embeddings = nn.Embedding(context_length, embedding_dim)
        # Stack of transformer blocks.
        self.transformer_blocks = nn.Sequential(*[TransformerBlock(embedding_dim, num_heads) for _ in range(num_blocks)])
        self.final_norm = nn.LayerNorm(embedding_dim)
        self.head = nn.Linear(embedding_dim, vocab_size)

    def forward(self, idx, targets=None):
        # idx: (B, T)
        B, T = idx.shape
        # Get token embeddings.
        tok_emb = self.token_embeddings(idx)  # (B, T, embedding_dim)
        # Create positional embeddings for each token position.
        pos_emb = self.position_embeddings(torch.arange(T, device=device))  # (T, embedding_dim)
        # Combine token and positional embeddings.
        x = tok_emb + pos_emb  # (B, T, embedding_dim)
        # Pass through transformer blocks and final layer norm.
        x = self.transformer_blocks(x)
        x = self.final_norm(x)
        # Compute logits over vocabulary.
        logits = self.head(x)  # (B, T, vocab_size)

        if targets is None:
            return logits, None

        # Reshape logits and targets for loss computation.
        logits = logits.view(B * T, vocab_size)
        targets = targets.view(B * T)
        loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate_text(self, idx, max_tokens):
        # Generate text autoregressively.
        for _ in range(max_tokens):
            idx_cond = idx[:, -context_length:]
            logits, _ = self(idx_cond)
            probs = F.softmax(logits[:, -1, :], dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, next_token), dim=1)
        return idx

In [ ]:
# DO NOT MODIFY ANY OF THIS CODE

# Initialize and train the model
model = TransformerLanguageModel().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=init_lr)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)
best_val_loss = float('inf')
no_progress = 0
max_patience = 10

for step in tqdm(range(max_iterations)):
    if step % log_interval == 0 or step == max_iterations - 1:
        current_losses = evaluate_loss()
        print(f"Step {step}: train loss {current_losses['train']:.4f}, val loss {current_losses['val']:.4f}")
        scheduler.step(current_losses['val'])

        if current_losses['val'] < best_val_loss:
            best_val_loss = current_losses['val']
            no_progress = 0
        else:
            no_progress += 1

    batch_x, batch_y = generate_batch('train')
    _, batch_loss = model(batch_x, batch_y)
    optimizer.zero_grad()
    batch_loss.backward()
    optimizer.step()

start_context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_output = decode_text(model.generate_text(start_context, max_tokens=1000)[0].tolist())

with open("output.txt", "w", encoding="utf-8") as out_file:
    out_file.write(generated_output)

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
  0%|          | 0/30000 [00:00<?, ?it/s]

Step 0: train loss 4.4109, val loss 4.4016


  1%|          | 204/30000 [00:15<2:39:27,  3.11it/s]

Step 200: train loss 2.4885, val loss 2.4840


  1%|▏         | 408/30000 [00:26<2:07:58,  3.85it/s]

...

Step 29600: train loss 1.4029, val loss 1.6123


 99%|█████████▉| 29806/30000 [17:50<00:24,  7.89it/s]

Step 29800: train loss 1.4037, val loss 1.6307


100%|██████████| 30000/30000 [17:57<00:00, 27.85it/s]

Step 29999: train loss 1.4001, val loss 1.5983


In [ ]:
# DO NOT MODIFY ANY OF THIS CODE

# Generate from the model
with torch.no_grad():
    context = torch.tensor(encode_text("JULIET: "), dtype=torch.long).unsqueeze(0).to(device)
    generated_text = decode_text(model.generate_text(context, max_tokens=200)[0].tolist())
    print(generated_text)

JULIET: where an excuse.

First Gewons: when he is too honour not
take all the were should
As be rest mine is fled: by now, infected great on the news
And friends abstand.I hast a cannot confession of your pl
